# Gathers data for CWFS unit tests
### Used during 2021-11-05
Using symmetric 0.8 nm offsets.

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

### Optional: Slew to a target
This is not part of the script but seems to come in handy, so it is here for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

Uncomment the following line to slew

In [ ]:
target = "HD 49849" 

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

## Grab focus offsets incase we need to come back into focus

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(original_focus_offset)

In [ ]:
original_offset_summary = await script.atcs.rem.ataos.evt_correctionOffsets.aget()
print(original_offset_summary)

## Offset Hexapod in x and y by 1.0 mm, and z by 0.05mm

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(x=1.0, y=1.0, z=0.05)

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

### Now print a PDF of this notebook, <br>
### do a git-pull on ts_externalscripts, <br>
### and restart and clear notebook!